In [ ]:
%pip install openai
%pip install rouge
%pip install rouge-score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 각 대화 상황을 위한 지시문 학습 (Instruction Tuning) 데이터셋 생성

### 1. SEED Task 만들기

In [ ]:
import json
path = '/content/drive/MyDrive/데이터/ko_alpaca_seedset_100.json'

with open(path, 'r') as f:
    seed_data = json.load(f)

In [ ]:
for k, v in seed_data[6].items():
  print(k, v)

instruction 다음 분수가 1/4에 해당하는 이유를 설명하세요.
input 4/16
output 분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.


### 2. insturction 생성을 위한 프롬프트 만들기

In [ ]:
prompt_for_instruction_generation = """
20개의 다양한 작업 지침 세트를 제시하라는 요청을 받습니다. 이러한 작업 지침은 GPT 모델에 제공되며 지침을 완료한 GPT 모델을 평가합니다.

요구 사항은 다음과 같습니다:
1. 다양성을 극대화하기 위해 각 지시에 대해 동사를 반복하지 않도록 하세요.
2. 지시에 사용되는 언어도 다양해야 합니다. 예를 들어, 질문과 명령형 지시를 결합해야 합니다.
3. 지시 사항의 유형이 다양해야 합니다. 목록에는 개방형 생성, 분류, 편집 등과 같은 다양한 유형의 작업이 포함되어야 합니다.
2. GPT 언어 모델은 지시를 완료할 수 있어야 합니다. 예를 들어, 어시스턴트에게 시각적 또는 음성 출력을 만들도록 요청하지 마세요. 또 다른 예로, 어시스턴트에게 오후 5시에 깨워달라고 요청하거나 어떤 작업도 수행할 수 없으므로 미리 알림을 설정해 달라고 요청하지 마세요.
3. 지침은 영어로 작성해야 합니다.
4. 지침은 1~2문장으로 작성합니다. 명령형 문장이나 질문도 허용됩니다.
5. 지침에 대한 적절한 입력을 생성해야 합니다. 입력 필드에는 지침에 대해 제공되는 구체적인 예가 포함되어야 합니다. 실제 데이터를 포함해야 하며 단순한 자리 표시자를 포함해서는 안 됩니다. 입력은 지시를 어렵게 만들 수 있는 상당한 내용을 제공해야 하지만 100단어를 넘지 않는 것이 좋습니다.

6. 모든 지시에 입력이 필요한 것은 아닙니다. 예를 들어 "세계에서 가장 높은 봉우리는 무엇인가"라는 일반적인 정보를 묻는 지시의 경우 구체적인 맥락을 제공할 필요는 없습니다. 이 경우 입력 필드에 "<noinput>"을 입력하기만 하면 됩니다.
7. 출력은 명령어와 입력에 대한 적절한 응답이어야 합니다. 출력은 100단어 미만이 되도록 하세요.

20개의 작업 목록:
"""

In [ ]:
system_prompt = """
아래는 작업을 설명하는 명령어입니다. 입력이 없으면 입력을 제외하고, 명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.
추가적 입력이 있다면 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력에 따라 요청을 적절히 완료하는 응답을 작성하세요.
예시로 주어진 텍스트들의 포멧을 잘 따라서 생성하도록 하세요.
"""

In [ ]:
import re
import random


def encode_prompt(prompt_instructions, prompt):
    """Encode multiple prompt instructions into a single string."""
    #prompt = open("./prompt.txt").read() + "\n"

    for idx, task_dict in enumerate(prompt_instructions):
        (instruction, input, output) = task_dict["instruction"], task_dict["input"], task_dict["output"]
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        input = "<noinput>" if input.lower() == "" else input
        prompt += f"###\n"
        prompt += f"{idx + 1}. Instruction: {instruction}\n"
        prompt += f"{idx + 1}. Input:\n{input}\n"
        prompt += f"{idx + 1}. Output:\n{output}\n"
    prompt += f"###\n"
    prompt += f"{idx + 2}. Instruction:"
    return prompt

def sample_seed_data(instructions):
  sampled_seed_data_index = random.sample(range(0,len(instructions)),10)
  sampled_seed_data = []
  for id in sampled_seed_data_index:
    sampled_seed_data.append(instructions[id])
  return sampled_seed_data


prompt = encode_prompt(sample_seed_data(seed_data), prompt_for_instruction_generation)
print(prompt)


20개의 다양한 작업 지침 세트를 제시하라는 요청을 받습니다. 이러한 작업 지침은 GPT 모델에 제공되며 지침을 완료한 GPT 모델을 평가합니다.

요구 사항은 다음과 같습니다:
1. 다양성을 극대화하기 위해 각 지시에 대해 동사를 반복하지 않도록 하세요.
2. 지시에 사용되는 언어도 다양해야 합니다. 예를 들어, 질문과 명령형 지시를 결합해야 합니다.
3. 지시 사항의 유형이 다양해야 합니다. 목록에는 개방형 생성, 분류, 편집 등과 같은 다양한 유형의 작업이 포함되어야 합니다.
2. GPT 언어 모델은 지시를 완료할 수 있어야 합니다. 예를 들어, 어시스턴트에게 시각적 또는 음성 출력을 만들도록 요청하지 마세요. 또 다른 예로, 어시스턴트에게 오후 5시에 깨워달라고 요청하거나 어떤 작업도 수행할 수 없으므로 미리 알림을 설정해 달라고 요청하지 마세요.
3. 지침은 영어로 작성해야 합니다.
4. 지침은 1~2문장으로 작성합니다. 명령형 문장이나 질문도 허용됩니다.
5. 지침에 대한 적절한 입력을 생성해야 합니다. 입력 필드에는 지침에 대해 제공되는 구체적인 예가 포함되어야 합니다. 실제 데이터를 포함해야 하며 단순한 자리 표시자를 포함해서는 안 됩니다. 입력은 지시를 어렵게 만들 수 있는 상당한 내용을 제공해야 하지만 100단어를 넘지 않는 것이 좋습니다.

6. 모든 지시에 입력이 필요한 것은 아닙니다. 예를 들어 "세계에서 가장 높은 봉우리는 무엇인가"라는 일반적인 정보를 묻는 지시의 경우 구체적인 맥락을 제공할 필요는 없습니다. 이 경우 입력 필드에 "<noinput>"을 입력하기만 하면 됩니다.
7. 출력은 명령어와 입력에 대한 적절한 응답이어야 합니다. 출력은 100단어 미만이 되도록 하세요.

20개의 작업 목록:
###
1. Instruction: 다음 문장을 편집하여 더 간결하게 만드세요.
1. Input:
그는 5분 후에 도착할 버스를 타기 위해 버스 정류장으로 달려갔습니다.
1. Output:
그는 버스 정류장으로 뛰어가서 5분 후에오는 

### 3. SEED instruction으로 새로운 instruction 생성하기

In [ ]:
from openai import OpenAI

OPENAI_API_KEY= # OpenAI API Key 입력
model_name="gpt-3.5-turbo"

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "이전 대화 기록은 모두 무시하세요.",
          }
      ],
      model=model_name,
  )

def generate_new_instruction(seed_data):
  sampled_seed_data = sample_seed_data(seed_data)
  prompt = encode_prompt(sampled_seed_data, prompt_for_instruction_generation)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": system_prompt,
          },
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model_name,
  )
  return chat_completion.choices[0].message.content

In [ ]:
new_instructions = generate_new_instruction(seed_data)
print(new_instructions)

1. 야간에 사진을 찍을 때 가장 좋은 설정은 무엇입니까?
2. Input:
<noinput>
2. Output:
야간에 사진을 찍을 때 가장 좋은 설정은 살짝 낮춘 조도, 적절한 셔터 스피드, 그리고 적당한 화이트 밸런스 조정입니다.
###
12. Instruction: 우주 비행사가 되기 위한 교육과정을 설명하세요.
12. Input:
<noinput>
12. Output:
우주 비행사가 되기 위한 교육과정은 항공 우주 공학, 물리학, 비행 훈련, 의료 검사, 우주에 대한 이론적 강의 등을 포함합니다.
###

14. Instruction: "사과"를 포함한 문장을 만들어보세요.
14. Input:
<noinput>
14. Output:
아침에 잘 익은 사과를 먹어서 기분이 좋습니다.
###
15. Instruction: 순환정신경망의 기능을 설명하세요.
15. Input:
<noinput>
15. Output:
순환신경망은 이전에 발생한 정보를 기억하고 현재 입력에 따라 다음 단어 또는 시퀀스를 예측하는 데 사용됩니다.
###
16. Instruction: 사진 속 인물의 감정을 분석하는 방법을 알려주세요.
16. Input:
<noinput>
16. Output:
사진 속 인물의 감정을 분석하는 방법으로는 얼굴 표정 인식 알고리즘을 사용하여 각 부분의 표정을 분석하고 감정을 파악하는 방법이 있습니다.
###
17. Instruction: 효율적인 공부 방법에 대해 설명해보세요.
17. Input:
<noinput>
17. Output:
효율적인 공부 방법에는 목표 설정, 계획 수립, 집중력 유지, 적절한 휴식, 주기적인 복습, 퀴즈 응시 등이 포함됩니다.
###
18. Instruction: 새로운 프로그램을 설치하는 방법을 알려주세요.
18. Input:
<noinput>
18. Output:
새로운 프로그램을 설치하는 방법은 공식 웹사이트나 앱 스토어를 통해 다운로드하거나, 수동으로 설치 파일을 다운로드하여 실행하는 방법이 있습니다.
###
19. 

In [ ]:
# 알파카 코드에서 수정
import string

def find_word_in_string(w, s):
    return re.compile(r"\b({0})\b".format(w), flags=re.IGNORECASE).search(s)

def post_process_gpt3_response(num_prompt_instructions, response):
    if response is None:
        return []
    raw_instructions = f"{num_prompt_instructions+1}. Instruction:" + response
    raw_instructions = re.split("###", raw_instructions)
    instructions = []
    for idx, inst in enumerate(raw_instructions):
        # if the decoding stops due to length, the last example is likely truncated so we discard it
        if idx == len(raw_instructions) - 1 :
            continue
        idx += num_prompt_instructions + 1
        splitted_data = re.split(f"{idx}\.\s+(Instruction|Input|Output):", inst)
        if len(splitted_data) != 7:
            continue
        else:
            inst = splitted_data[2].strip()
            input = splitted_data[4].strip()
            input = "" if input.lower() == "<noinput>" else input
            output = splitted_data[6].strip()
        # filter out too short or too long instructions
        if len(inst.split()) <= 3 or len(inst.split()) > 150:
            continue


        # filter based on keywords that are not suitable for language models.
        blacklist = [
            "이미지",
            "그림",
            "그래프",
            "차트",
            "사진",
            "파일",
            "지도",
            "비디오",
            "오디오",
            "음악",
            "소리",
            "순서도",
        ]
        blacklist += []
        if any(find_word_in_string(word, inst) for word in blacklist):
            continue
        # We found that the model tends to add "write a program" to some existing instructions, which lead to a lot of such instructions.
        # And it's a bit comfusing whether the model need to write a program or directly output the result.
        # Here we filter them out.
        # Note this is not a comprehensive filtering for all programming instructions.
        if inst.startswith("Write a program"):
            continue
        # filter those starting with punctuation
        if inst[0] in string.punctuation:
            continue
        instructions.append({"instruction": inst, "input": input, "output": output})
    return instructions

processed_instruction = post_process_gpt3_response(10, new_instructions)
print(processed_instruction)


[{'instruction': '우주 비행사가 되기 위한 교육과정을 설명하세요.', 'input': '', 'output': '우주 비행사가 되기 위한 교육과정은 항공 우주 공학, 물리학, 비행 훈련, 의료 검사, 우주에 대한 이론적 강의 등을 포함합니다.'}]


### 4. Rouge score 필터링

In [ ]:
from rouge import Rouge

model_out = [
"사과를 매일 아침에 먹는 것이 건강에 좋다고 들었어요.",
"사과를 매일 아침에 먹는 것이 건강에 좋다고 들었어요.",
]

reference = [
"사과를 튀겨서 간식으로 먹는 것도 좋은 아이디어입니다.",
"사과를 매일 저녁에 먹는 것이 건강에 좋다고 들었어요.",
]
rouge = Rouge()
score = rouge.get_scores(model_out, reference)

print(score[0])
print(score[1])

{'rouge-1': {'r': 0.2857142857142857, 'p': 0.25, 'f': 0.266666661688889}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.2857142857142857, 'p': 0.25, 'f': 0.266666661688889}}
{'rouge-1': {'r': 0.875, 'p': 0.875, 'f': 0.874999995}, 'rouge-2': {'r': 0.7142857142857143, 'p': 0.7142857142857143, 'f': 0.7142857092857143}, 'rouge-l': {'r': 0.875, 'p': 0.875, 'f': 0.874999995}}


In [ ]:
def filtering_generated_instructions(processed_new_instruction, seed_data):
  for new_inst in processed_new_instruction:
    insturction = new_inst['instruction']

    is_similar = False
    for seed in seed_data:
      exist_instruction = seed['instruction']
      score = rouge.get_scores([insturction], [exist_instruction])
      if score[0]['rouge-1']['r'] > 0.7 or score[0]['rouge-2']['r'] > 0.7 :
        is_similar = True
        continue

    if is_similar:
      continue
    seed_data.append(new_inst)

  return seed_data

new_instruction_set = filtering_generated_instructions(processed_instruction, seed_data)

In [ ]:
## new instructions
print(seed_data[100:])

[{'instruction': '우주 비행사가 되기 위한 교육과정을 설명하세요.', 'input': '', 'output': '우주 비행사가 되기 위한 교육과정은 항공 우주 공학, 물리학, 비행 훈련, 의료 검사, 우주에 대한 이론적 강의 등을 포함합니다.'}]


**reference**

https://github.com/tatsu-lab/stanford_alpaca